# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [2]:
# Load the libraries as required.
import pandas as pd
import numpy as np
import pickle
import shap
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PowerTransformer

c:\Users\ifara\miniconda3\envs\faraz_shahid\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load data
fires_dt = pd.read_csv('C:/Users/ifara/OneDrive/Desktop/DSI/Programming/production/05_src/data/fires/forestfires.csv')
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('C:/Users/ifara/OneDrive/Desktop/DSI/Programming/production/05_src/data/fires/forestfires.csv', header = 0, names = columns))
print(fires_dt.info())
print(fires_dt.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB
None
   coord_x  coord_y month  day  ffmc   dmc     dc  isi  temp  rh  wind  rain  \
0        7        5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0   
1        7        4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0   
2        7       

# Get X and Y

Create the features data frame and target data.

In [4]:
X = fires_dt.drop(columns=['area'])
y = fires_dt['area']

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [5]:
# Identify numeric and categorical columns
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Preproc1: Scaling + One-hot encoding
preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [6]:
# Preproc2: Scaling + Non-linear transformation + One-hot encoding
preproc2 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('non_linear', PowerTransformer(), numeric_cols),  
        ('cat', OneHotEncoder(), categorical_cols)
    ])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [7]:
# Pipeline A = preproc1 + baseline
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
pipeline_a = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', LinearRegression())
])

In [8]:
# Pipeline B = preproc2 + baseline
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
pipeline_b = Pipeline([
    ('preprocessing', preproc2),  # Preproc2 includes scaling + non-linear transformation
    ('regressor', LinearRegression())
])


In [9]:
# Pipeline C = preproc1 + advanced model
from sklearn.ensemble import RandomForestRegressor
pipeline_c = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', RandomForestRegressor(random_state=42))
])

In [10]:
# Pipeline D = preproc2 + advanced model
from sklearn.ensemble import RandomForestRegressor
pipeline_d = Pipeline([
    ('preprocessing', preproc2),  # Preproc2 includes scaling + non-linear transformation
    ('regressor', RandomForestRegressor(random_state=42))
])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [11]:
# Pipeline A: No hyperparameters to tune for Linear Regression
param_grid_a = {}

In [12]:
# Pipeline B: Tune preproc2 options (e.g., PowerTransformer method)
param_grid_b = {
    'preprocessing__non_linear__method': ['yeo-johnson', 'box-cox'] 
}

In [13]:
# Pipeline C: Tune Random Forest hyperparameters
param_grid_c = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20]
}

In [14]:
# Pipeline D: Tune Random Forest hyperparameters
param_grid_d = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20]
}

In [17]:
# Perform GridSearchCV for all pipelines
grid_search_a = GridSearchCV(pipeline_a, param_grid_a, cv=5, scoring='neg_mean_squared_error')
grid_search_b = GridSearchCV(pipeline_b, param_grid_b, cv=5, scoring='neg_mean_squared_error')
grid_search_c = GridSearchCV(pipeline_c, param_grid_c, cv=5, scoring='neg_mean_squared_error')
grid_search_d = GridSearchCV(pipeline_d, param_grid_d, cv=5, scoring='neg_mean_squared_error')

# Fit the models
grid_search_a.fit(X, y)
grid_search_b.fit(X, y)
grid_search_c.fit(X, y)
grid_search_d.fit(X, y)

print("Pipeline A - Best parameters:", grid_search_a.best_params_)
print("Pipeline A - Best score:", -grid_search_a.best_score_)

print("Pipeline B - Best parameters:", grid_search_b.best_params_)
print("Pipeline B - Best score:", -grid_search_b.best_score_)

print("Pipeline C - Best parameters:", grid_search_c.best_params_)
print("Pipeline C - Best score:", -grid_search_c.best_score_)

print("Pipeline D - Best parameters:", grid_search_d.best_params_)
print("Pipeline D - Best score:", -grid_search_d.best_score_)

# Evaluate all pipelines
def evaluate_pipeline(grid_search, X, y):
    y_pred = grid_search.predict(X)
    mse = mean_squared_error(y, y_pred)  # Compute MSE
    rmse = mse ** 0.5  # Compute RMSE manually
    mae = mean_absolute_error(y, y_pred)  # Compute MAE
    return rmse, mae

# Evaluate Pipeline A
rmse_a, mae_a = evaluate_pipeline(grid_search_a, X, y)
print(f"Pipeline A - RMSE: {rmse_a}, MAE: {mae_a}")

# Evaluate Pipeline B
rmse_b, mae_b = evaluate_pipeline(grid_search_b, X, y)
print(f"Pipeline B - RMSE: {rmse_b}, MAE: {mae_b}")

# Evaluate Pipeline C
rmse_c, mae_c = evaluate_pipeline(grid_search_c, X, y)
print(f"Pipeline C - RMSE: {rmse_c}, MAE: {mae_c}")

# Evaluate Pipeline D
rmse_d, mae_d = evaluate_pipeline(grid_search_d, X, y)
print(f"Pipeline D - RMSE: {rmse_d}, MAE: {mae_d}")

c:\Users\ifara\miniconda3\envs\faraz_shahid\lib\site-packages\sklearn\model_selection\_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\ifara\miniconda3\envs\faraz_shahid\lib\site-packages\sklearn\model_selection\_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\ifara\miniconda3\envs\faraz_shahid\lib\site-packages\sklearn\metrics\_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\ifara\miniconda3\envs\faraz_shahid\lib\site-packages\sklearn\metrics\_scorer.py", line 380, in _score
    y_pred = method_caller(
  File "c:\Users\ifara\miniconda3\envs\faraz_shahid\lib\site-packages\sklearn\metrics\_scorer.py", line 90, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users\ifara\minicond

Pipeline A - Best parameters: {}
Pipeline A - Best score: nan
Pipeline B - Best parameters: {'preprocessing__non_linear__method': 'yeo-johnson'}
Pipeline B - Best score: nan
Pipeline C - Best parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 50}
Pipeline C - Best score: nan
Pipeline D - Best parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 50}
Pipeline D - Best score: nan
Pipeline A - RMSE: 62.12143311792724, MAE: 20.298523994757943
Pipeline B - RMSE: 61.844434381836024, MAE: 20.433190151731363
Pipeline C - RMSE: 25.746844165044806, MAE: 8.277227362991619
Pipeline D - RMSE: 25.83670421905488, MAE: 8.182377669706181


# Evaluate

+ Which model has the best performance?
Pipeline C is the best since it has the lowest RMSE

# Export

+ Save the best performing model to a pickle file.

In [19]:
# Save the best-performing model
import pickle

# Compare scores and select the best model
best_score = min(-grid_search_a.best_score_, -grid_search_b.best_score_, 
                 -grid_search_c.best_score_, -grid_search_d.best_score_)

if best_score == -grid_search_a.best_score_:
    best_model = grid_search_a.best_estimator_
elif best_score == -grid_search_b.best_score_:
    best_model = grid_search_b.best_estimator_
elif best_score == -grid_search_c.best_score_:
    best_model = grid_search_c.best_estimator_
else:
    best_model = grid_search_d.best_estimator_

# Save the best model
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

print("Best model saved as 'best_model.pkl'.")

Best model saved as 'best_model.pkl'.


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [23]:
import pickle
import shap
from sklearn.model_selection import train_test_split

with open('best_model.pkl', 'rb') as f:
    best_model = pickle.load(f)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Extract the preprocessed data
preprocessed_X_test = best_model.named_steps['preprocessing'].transform(X_test)

# Create a SHAP explainer for the model
explainer = shap.Explainer(best_model.named_steps['regressor'])
shap_values = explainer(preprocessed_X_test)

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.